In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci
from Wick import *
from math import factorial, sqrt

In [2]:
norbs = 3
Nocc = 2
vacuumOccupations = [1 for i in range(Nocc)] + [0 for i in range (norbs - Nocc)]
print(vacuumOccupations)

[1, 1, 0]


In [3]:
oP = operatorProduct([basicOperator(2,0,1), basicOperator(1,0,1), basicOperator(2,1,1), basicOperator(1,1,1)])
nOP = normalOrder(oP, vacuumOccupations)
print(oP)
print(nOP)
print(wickExpand(oP, vacuumOccupations))
print(vacuumExpectationValue(oP, vacuumOccupations))

1.0 * a_{2\alpha}a_{1\alpha}a^{2\alpha}a^{1\alpha}
1.0 * a_{1\alpha}a^{2\alpha}a_{2\alpha}a^{1\alpha}
0
-1.0 * a_{1\alpha}a^{1\alpha}
0
0
0
0
0
0
0
-1.0 * a_{1\alpha}a^{1\alpha}
 + 1.0 * a_{1\alpha}a^{2\alpha}a_{2\alpha}a^{1\alpha}
0.0


In [4]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [5]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [6]:
eri

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [7]:
mf.energy_elec() + mf.energy_nuc()

array([-1.10263881,  1.27706243])

In [8]:
mf.mo_coeff

array([[ 0.56092571, -1.10314488],
       [ 0.56092571,  1.10314488]])

In [9]:
mf.nelec

(1, 1)

In [10]:
def spinFreeSingleExcitation(p, q):
    summandList = []
    for spin in range(2):
        summandList.append(operatorProduct([basicOperator(p,1,spin), basicOperator(q,0,spin)]))
    return operatorSum(summandList)

In [11]:
def spinFreeDoubleExcitation(p, q, r, s):
    summandList = []
    for spin1 in range(2):
        for spin2 in range(2):
            summandList.append(operatorProduct([basicOperator(p,1,spin2), basicOperator(q,1,spin1), basicOperator(s,0,spin1), basicOperator(r,0,spin2)]))
    return operatorSum(summandList)

In [12]:
mf.mol.nao

2

In [13]:
Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1 for i in range(Nocc)] + [0 for i in range (Norbs - Nocc)]


In [14]:
hamiltonian1Body = operatorSum([])
for p in range(Norbs):
    for q in range(Norbs):
        hamiltonian1Body = hamiltonian1Body + h1[p, q] * spinFreeSingleExcitation(p, q)

In [15]:
print(hamiltonian1Body)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}


In [16]:
print(wickExpand(hamiltonian1Body, [1,0]))

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0
-2.379701237214037
 + 1.1898506186070186 * a_{0\alpha}a^{0\alpha}
 + 1.1898506186070186 * a_{0\beta}a^{0\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [17]:
hamiltonian1Body

In [18]:
wickExpand(hamiltonian1Body, [1,0])

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0


In [19]:
for s in hamiltonian1Body.summandList:
    print(s)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
-1.1898506186070186 * a^{0\beta}a_{0\beta}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-0.5337491016607854 * a^{1\beta}a_{1\beta}


In [20]:
for s in normalOrder(hamiltonian1Body, [1,0]).summandList:
    print(s)

1.1898506186070186 * a_{0\alpha}a^{0\alpha}
1.1898506186070186 * a_{0\beta}a^{0\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-0.5337491016607854 * a^{1\beta}a_{1\beta}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [21]:
for s in wickExpand(hamiltonian1Body, [1,0]).summandList:
    print(s)

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0
-2.379701237214037
1.1898506186070186 * a_{0\alpha}a^{0\alpha}
1.1898506186070186 * a_{0\beta}a^{0\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-0.5337491016607854 * a^{1\beta}a_{1\beta}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [22]:
w = wickExpand(hamiltonian1Body, [1,0])

-1.1898506186070186
0
-1.1898506186070186
0
0
0
0
0


In [23]:
w

In [24]:
print(w)

-2.379701237214037
 + 1.1898506186070186 * a_{0\alpha}a^{0\alpha}
 + 1.1898506186070186 * a_{0\beta}a^{0\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}


In [25]:
vacuumExpectationValue(hamiltonian1Body, [1,0])

-2.379701237214037

In [26]:
vEV = 0.
for summand in w.summandList:
    if summand.operatorList == []:
        print(summand)
        vEV += summand.prefactor


-2.379701237214037


In [27]:
vEV

-2.379701237214037

In [28]:
mf.energy_nuc()

0.6230529146661897

In [29]:
mf.energy_elec()

(-1.7256917258579134, 0.6540095113561237)

In [30]:
hamiltonian2Body = operatorSum([])
for p in range(Norbs):
    for q in range(p, Norbs):
        for r in range(Norbs):
            for s in range(r, Norbs):
                x = int(q + Norbs * p - 0.5 * p * (p + 1))
                y = int(s + Norbs * r - 0.5 * r * (r + 1))
                hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)

In [31]:
hamiltonian2Body = operatorSum([])
for p in range(Norbs):
    for q in range(Norbs):
        for r in range(p):
            for s in range(q):
                if r >= p:
                    x = int(q + Norbs * p - 0.5 * p * (p + 1))
                else:
                    x = int(p + Norbs * r - 0.5 * r * (r + 1))
                if q >= s:
                    y = int(s + Norbs * r - 0.5 * r * (r + 1))
                else:
                    y = int(q + Norbs * s - 0.5 * s * (s + 1))
                hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)

In [207]:
print(eri)

[[6.54009511e-01 8.32667268e-17 6.45249427e-01]
 [1.04083409e-16 1.87521981e-01 5.55111512e-17]
 [6.45249427e-01 1.38777878e-16 6.78136184e-01]]


In [32]:
hamiltonian2Body = operatorSum([])
for p in range(Norbs):
    for q in range(Norbs):
        for r in range(p + 1):
            for s in range(q + 1):
                x = int(p + Norbs * r - 0.5 * r * (r + 1))
                y = int(q + Norbs * s - 0.5 * s * (s + 1))
                print(p, q, r, s)
                print(x, y)
                if p == r and q == s:
                    hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)
                else:
                    hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s) + 0.5 * np.conjugate(eri[x, y]) * spinFreeDoubleExcitation(r, s, p, q)


0 0 0 0
0 0
0 1 0 0
0 1
0 1 0 1
0 2
1 0 0 0
1 0
1 0 1 0
2 0
1 1 0 0
1 1
1 1 0 1
1 2
1 1 1 0
2 1
1 1 1 1
2 2


In [33]:
print(eri)

[[6.54009511e-01 8.32667268e-17 6.45249427e-01]
 [1.04083409e-16 1.87521981e-01 5.55111512e-17]
 [6.45249427e-01 1.38777878e-16 6.78136184e-01]]


In [34]:
print(hamiltonian2Body)

0.3270047556780619 * a^{0\alpha}a^{0\alpha}a_{0\alpha}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\alpha}a_{1\alpha}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\alpha}a_{1\alpha}a_{1\alpha}
 + 0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3270047556780619 *

In [35]:
#for s in wickExpand(hamiltonian2Body, [1,0]).summandList:
 #   print(s)

In [36]:
vacuumExpectationValue(hamiltonian2Body, [1,0])

0.6540095113561238

In [39]:
vacuumExpectationValue(hamiltonian2Body, [0,1])

0.6781361836853057

In [40]:
vacuumExpectationValue(hamiltonian1Body, [1,0]) + vacuumExpectationValue(hamiltonian2Body, [1,0])

-1.7256917258579132

In [41]:
vacuumExpectationValue(hamiltonian2Body, [1,0])

0.6540095113561238

In [42]:
mf.energy_elec()

(-1.7256917258579134, 0.6540095113561237)

In [43]:
hamiltonian = hamiltonian1Body + hamiltonian2Body

In [204]:
print(hamiltonian)

-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 0.0
 + 5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.093760990451354

In [206]:
print(hamiltonian2Body)

0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3270047556780619 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
 + 5.204170427930421e-17 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{1\beta}
 + 4.163336342344337e-17 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{0\beta}
 + 0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 

In [44]:
vacuumExpectationValue(hamiltonian, [1,0])

-1.7256917258579132

In [45]:
singlesAmplitudes = np.ones((Nocc, Norbs - Nocc))

In [197]:
doublesAmplitudes = np.ones((Nocc, Nocc, Norbs-Nocc, Norbs-Nocc))

In [198]:
doublesAmplitudes

array([[[[1.]]]])

In [199]:
newdoublesAmplitudes = deepcopy(doublesAmplitudes)
newdoublesAmplitudes[0,0,0,0] = -0.136380375 *0.5 / 0.990656547


In [49]:
clusterSingles = operatorSum([])
for a in range(Nocc):
    for k in range(Norbs - Nocc):
        r = k + Nocc
        clusterSingles = clusterSingles + singlesAmplitudes[a,k] * spinFreeSingleExcitation(r, a)

In [200]:
clusterDoubles = operatorSum([])
for a in range(Nocc):
    for b in range(Nocc):
        for k in range(Norbs - Nocc):
            r = k + Nocc
            for l in range(Norbs - Nocc):
                s = l + Nocc
                clusterDoubles = clusterDoubles + doublesAmplitudes[a,b,k,l] * spinFreeDoubleExcitation(r, s, a, b)
clusterDoubles.checkNilpotency()

In [51]:
newclusterDoubles = operatorSum([])
for a in range(Nocc):
    for b in range(Nocc):
        for k in range(Norbs - Nocc):
            r = k + Nocc
            for l in range(Norbs - Nocc):
                s = l + Nocc
                newclusterDoubles = newclusterDoubles + newdoublesAmplitudes[a,b,k,l] * spinFreeDoubleExcitation(r, s, a, b)
newclusterDoubles.checkNilpotency()

In [201]:
print(newclusterDoubles)

0.0
 + -0.06883332846938728 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332846938728 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [53]:
print(clusterSingles.summandList[0] * clusterDoubles.summandList[0])


0.0 * a^{1\alpha}a_{0\alpha}


In [202]:
print(clusterSingles)
print(clusterDoubles)
print(hamiltonian)

1.0 * a^{1\alpha}a_{0\alpha}
 + 1.0 * a^{1\beta}a_{0\beta}
0.0
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 0.0
 + 5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\

In [55]:
hamiltonian.checkNilpotency()
print(hamiltonian)

-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 0.0
 + 5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.093760990451354

In [203]:
projectionDoubles = clusterDoubles.conjugate()

In [56]:
print(clusterSingles * hamiltonian * clusterSingles)

0.0
 + 0.0 * a^{1\alpha}a_{0\alpha}
 + 0.0 * a^{1\beta}a_{0\beta}
 + 0.0 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.0 * a^{1\beta}a_{0\beta}a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{1\beta}a^{1\beta}a_{0\beta}
 + -1.1898506186070186 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{1\alpha}a^{1\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{1\alpha}a_{0\alpha}a^{0\beta}a_{1\beta}a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{0\beta}a^{1\alpha}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{1\beta}a_{0\beta}a^{0\alpha}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + -1.1898506186070186 * a^{1\beta}a_{0\beta}a^{0\beta}a_{0\beta}a^{1\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{1\beta}a_{0\beta}a^{0\beta}a_{1\beta}a^{1\alpha}a_{0\alpha}
 + 2.60021255284

In [57]:
def exponentialOperator(operator, maxOrder):
    exponential = operatorSum([operatorProduct([], 1.)])
    for k in range(maxOrder):
        exponential += (1 / factorial(k + 1)) * pow(operator, k + 1)
    return exponential

In [58]:
print(exponentialOperator(clusterDoubles, 3))

1.0
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


In [59]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [60]:
print(commutator(clusterSingles, clusterSingles))

0.0
 + 0.0 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.0 * a^{1\beta}a_{0\beta}a^{1\alpha}a_{0\alpha}


In [61]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    result.checkNilpotency()
    return result

In [180]:
print(hamiltonian1Body)
print(clusterSingles)
print(exponentialOperator((-1) * clusterSingles, 3) * hamiltonian1Body * exponentialOperator(clusterSingles, 3))
print(BCHSimilarityTransform(hamiltonian1Body, clusterSingles, 3))
print(exponentialOperator((-1) * clusterSingles, 4) * hamiltonian1Body * exponentialOperator(clusterSingles, 4) + (-1) * BCHSimilarityTransform(hamiltonian1Body, clusterSingles, 4))

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
1.0 * a^{1\alpha}a_{0\alpha}
 + 1.0 * a^{1\beta}a_{0\beta}
0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + 0.0 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + 0.0 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 4.7142356915405136e-17 * a^

In [63]:
print(BCHSimilarityTransform(hamiltonian, clusterSingles, 1))

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.0 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.0 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}a^{1\beta}a_{0\beta}
 + 0.5337491016607854 * a^{1\alpha}a_{0\alpha}a^{1\beta

In [181]:
bch = BCHSimilarityTransform(hamiltonian, clusterDoubles, 4)
bch.checkNilpotency()
print(bch)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.09376099045135429 * a^{1\beta}a^{1\alpha}a_{0\alph

In [192]:
print(clusterDoubles)

0.0
 + -0.06883332874447325 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332874447325 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [182]:
newbch = BCHSimilarityTransform(hamiltonian, newclusterDoubles, 4)
newbch.checkNilpotency()
print(newbch)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.09376099045135429 * a^{1\beta}a^{1\alpha}a_{0\alph

In [67]:
for s in bch.summandList:
    print(s)
    print(vacuumExpectationValue(s, vacuum))


0.0
0.0
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.0
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}
-0.0
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-0.0
-0.5337491016607854 * a^{1\beta}a_{1\beta}
-0.0
-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
-1.1898506186070186
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
0.0
-1.1898506186070186 * a^{0\beta}a_{0\beta}
-1.1898506186070186
2.600212552842601e-17 * a^{0\beta}a_{1\beta}
0.0
0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0
6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
0
0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0
2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
0
0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
0
0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
0
5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
0
0.3226247132942422 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
0
6.938893903907228e-17 * a^{1\alph

In [68]:
print(bch.summandList[31])

0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}


In [193]:
energyEquation = vacuumExpectationValue(bch, vacuum, printing=True)
print(energyEquation)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
-1.1898506186070186 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
-0.006453881079145487 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881079145487 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.006453881079145487 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881079145487 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-1.751507250174495


In [189]:
newenergyEquation = vacuumExpectationValue(newbch, vacuum, printing=True)
print(newenergyEquation)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
-1.1898506186070186 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
-0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.006453881053353154 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881053353154 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-1.751507250071326


In [188]:
print(clusterDoubles)

0.0
 + -0.06883332874447325 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332874447325 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [187]:
print(clusterDoubles.conjugate())

0.0
 + -0.06883332874447325 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + -0.06883332874447325 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}


In [183]:
projected = clusterDoubles.conjugate() * bch
projected.checkNilpotency()
print(projected)

0.0
 + -0.0 * a^{1\alpha}a_{0\alpha}
 + -0.0 * a^{1\alpha}a_{1\alpha}
 + -0.0 * a^{1\beta}a_{0\beta}
 + -0.0 * a^{1\beta}a_{1\beta}
 + -0.0 * a^{0\alpha}a_{0\alpha}
 + 0.0 * a^{0\alpha}a_{1\alpha}
 + -0.0 * a^{0\beta}a_{0\beta}
 + 0.0 * a^{0\beta}a_{1\beta}
 + -0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + -0.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{1\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a

In [184]:
newprojected = newclusterDoubles.summandList[1].conjugate() * newbch
newprojected.checkNilpotency()
print(newprojected)

0.0
 + -0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 3.2449653383791726e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + 0.036739727234857224 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}
 + 3.2449653383791726e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{0\beta}
 + 0.036739727234857224 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}
 + -0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -4.7762716330187526e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + -1.910508653207501e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + -0.023339185339293696 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + -0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + -4.7762716330

In [77]:
#print(getPositionsForMultipleContraction(newprojected.summandList[42], 8))
#print(sumNFoldContractions(newprojected.summandList[42], 8))

In [185]:
projectedAmplitudeEquation = vacuumExpectationValue(projected, vacuum, printing=True)
print(projectedAmplitudeEquation)

-0.006453881079145487 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881079145487 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.006453881079145487 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881079145487 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.005637544530698165 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}
0.005637544530698165 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a_{0\beta}
-0.0025289177328457874 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.0025289177328457874 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.005637544530698165 * a^{0\alpha}a^{0\bet

In [186]:
newprojectedAmplitudeEquation = vacuumExpectationValue(newprojected, vacuum, printing=True)
print(newprojectedAmplitudeEquation)

-0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.005637544485638325 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}
0.005637544485638325 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a_{0\beta}
-0.0025289177126326208 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.0025289177126326208 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.005637544485638325 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\alpha}a_{0\alpha}
0.005637544485638325 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\beta}a_{0\beta}
-0

In [174]:
hartreeFockElectronicEnergy = vacuumExpectationValue(hamiltonian, vacuum)
hartreeFockEnergy = hartreeFockElectronicEnergy + mol.energy_nuc()
print(hartreeFockEnergy)

-1.1026388111917234


In [84]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [86]:
vacuumExpectationValue((1. + (-1.) * clusterDoubles) * hamiltonian * (1. + clusterDoubles), vacuum)

-1.3506477640524959

In [87]:
HT2 = hamiltonian * clusterDoubles
HT2.checkNilpotency()
print(HT2)

0.0
 + -0.0 * a^{1\alpha}a_{0\alpha}
 + -0.0 * a^{1\alpha}a_{1\alpha}
 + -0.0 * a^{1\beta}a_{0\beta}
 + -0.0 * a^{1\beta}a_{1\beta}
 + -0.0 * a^{0\alpha}a_{0\alpha}
 + 0.0 * a^{0\alpha}a_{1\alpha}
 + -0.0 * a^{0\beta}a_{0\beta}
 + 0.0 * a^{0\beta}a_{1\beta}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{1\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a^{1\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{1\beta}a^{0\alpha}a_{0\alpha}a_{1\beta}
 + 0.0 * a^{

In [89]:
HT2CorrelationEnergy = vacuumExpectationValue(HT2, vacuum)

In [90]:
print(HT2CorrelationEnergy)

0.37504396180541716


In [91]:
ECCD = hartreeFockElectronicEnergy + HT2CorrelationEnergy

In [92]:
ECCD

-1.350647764052496

In [94]:
vacuumExpectationValue(hamiltonian, vacuum)

-1.7256917258579132

In [95]:
newenergyEquation + mol.energy_nuc()

-1.1284543354051362

In [96]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [97]:
def contract2operators(o1, o2):
    if o1.quasi_cre_ann:
        return 0
    elif o2.quasi_cre_ann and (o1.spin == o2.spin):
        return int(o1.orbital == o2.orbital)
    else:
        return 0

In [98]:
def recursiveFullContraction(operatorProduct_, vacuum):
    operatorProduct_.applyFermiVacuum(vacuum)
    operatorList_ = operatorProduct_.operatorList
    if len(operatorList_) == 0:
        result = operatorProduct_.prefactor
    elif len(operatorList_) == 2:
        result = operatorProduct_.prefactor * contract2operators(operatorList_[0], operatorList_[1])
    elif len(operatorList_) % 2 == 0:
        result = 0
        for i in range(1, len(operatorList_) - 1):
            if contract2operators(operatorList_[0], operatorList_[i]):
                result += pow(-1, i-1) * recursiveFullContraction(operatorProduct(operatorList_[1:i] + operatorList_[i+1:], operatorProduct_.prefactor), vacuum)
            else:
                result += 0
        if contract2operators(operatorList_[0], operatorList_[-1]):
            result += recursiveFullContraction(operatorProduct(operatorList_[1:-1], operatorProduct_.prefactor), vacuum)
        else:
            result += 0
    return result
        


In [99]:
print(newprojected.summandList[42])
print(recursiveFullContraction(newprojected.summandList[42], vacuum))

-1.9726000450475795e-19 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\beta}a^{0\alpha}a_{1\alpha}a_{0\beta}
0.0


In [100]:
recursivehartreeFockElectronicEnergy = 0
for product in hamiltonian.summandList:
    recursivehartreeFockElectronicEnergy += recursiveFullContraction(product, vacuum)
recursivehartreeFockEnergy = recursivehartreeFockElectronicEnergy + mol.energy_nuc()
print(recursivehartreeFockEnergy)

-1.1026388111917234


In [101]:
newprojected = newclusterDoubles.summandList[1].conjugate() * newbch
newprojected.checkNilpotency()
print(newprojected)

0.0
 + -0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 3.2449653383791726e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + 0.036739727234857224 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}
 + 3.2449653383791726e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{0\beta}
 + 0.036739727234857224 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}
 + -0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -4.7762716330187526e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + -1.910508653207501e-18 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + -0.023339185339293696 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + -0.006453881053353154 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + -4.7762716330

In [102]:
recursiveFullContractedNewprojected = 0
for product in newprojected.summandList:
    recursiveFullContractedNewprojected += recursiveFullContraction(product, vacuum)
print(recursiveFullContractedNewprojected)

-5.256230352227234e-11


In [103]:
print(newclusterDoubles)

0.0
 + -0.06883332846938728 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332846938728 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [104]:
def getClusterDoubles(doublesAmplitudes_, Nocc_, Norbs_):
    clusterDoubles = operatorSum([])
    for a in range(Nocc_):
        for b in range(Nocc_):
            for k in range(Norbs_ - Nocc_):
                r = k + Nocc_
                for l in range(Norbs_ - Nocc_):
                    s = l + Nocc_
                    clusterDoubles = clusterDoubles + doublesAmplitudes_[a,b,k,l] * spinFreeDoubleExcitation(r, s, a, b)
    clusterDoubles.checkNilpotency()
    return clusterDoubles

In [105]:
def getClusterSingles(singlesAmplitudes_, Nocc_, Norbs_):
    clusterSingles = operatorSum([])
    for a in range(Nocc_):
        for k in range(Norbs_ - Nocc_):
            r = k + Nocc_
            clusterSingles = clusterSingles + singlesAmplitudes_[a,k] * spinFreeSingleExcitation(r, a)
    clusterSingles.checkNilpotency()
    return clusterSingles

In [106]:
Nocc1 = 2
Norbs1 = 4
singlesAmplitudes1 = np.zeros((Nocc1, Norbs1-Nocc1))
doublesAmplitudes1 = np.zeros((Nocc1, Nocc1, Norbs1-Nocc1, Norbs1-Nocc1))

In [107]:
flatSinglesAmplitudes = []
singlesExcitations1 = []
for a in range(Nocc1):
    for r in range(Nocc1, Norbs1):
        print(a,r)
        flatSinglesAmplitudes.append(0.)
        singlesExcitations1.append(spinFreeSingleExcitation(r, a))

0 2
0 3
1 2
1 3


In [108]:
flatDoublesAmplitudes = []
doublesExcitations1 = []
for a in range(Nocc1):
    for b in range(Nocc1):
        for r in range(Nocc1, Norbs1):
            for s in range(Nocc1, Norbs1):
                print(a,b,r,s)
                flatDoublesAmplitudes.append(0.)
                doublesExcitations1.append(spinFreeDoubleExcitation(r,s,a,b))

0 0 2 2
0 0 2 3
0 0 3 2
0 0 3 3
0 1 2 2
0 1 2 3
0 1 3 2
0 1 3 3
1 0 2 2
1 0 2 3
1 0 3 2
1 0 3 3
1 1 2 2
1 1 2 3
1 1 3 2
1 1 3 3


In [109]:
import itertools

In [110]:
def spinFreeNtupleExcitation(creTuple, annTuple):
    N = len(creTuple)
    if len(annTuple) != N:
        print("particle number not conserved")
        return 0
    summandList = []
    spinCombs = itertools.product(range(2), repeat=N)
    for spinComb in spinCombs:
        productList = []
        for o in range(N):
            productList = [basicOperator(creTuple[o], 1, spinComb[o])] + productList + [basicOperator(annTuple[o], 0, spinComb[o])]
        summandList.append(operatorProduct(productList))
    return operatorSum(summandList)

In [111]:
spFE = spinFreeNtupleExcitation((2,2,3), (0,1,1))
print(spFE)

1.0 * a^{3\alpha}a^{2\alpha}a^{2\alpha}a_{0\alpha}a_{1\alpha}a_{1\alpha}
 + 1.0 * a^{3\alpha}a^{2\alpha}a^{2\beta}a_{0\beta}a_{1\alpha}a_{1\alpha}
 + 1.0 * a^{3\alpha}a^{2\beta}a^{2\alpha}a_{0\alpha}a_{1\beta}a_{1\alpha}
 + 1.0 * a^{3\alpha}a^{2\beta}a^{2\beta}a_{0\beta}a_{1\beta}a_{1\alpha}
 + 1.0 * a^{3\beta}a^{2\alpha}a^{2\alpha}a_{0\alpha}a_{1\alpha}a_{1\beta}
 + 1.0 * a^{3\beta}a^{2\alpha}a^{2\beta}a_{0\beta}a_{1\alpha}a_{1\beta}
 + 1.0 * a^{3\beta}a^{2\beta}a^{2\alpha}a_{0\alpha}a_{1\beta}a_{1\beta}
 + 1.0 * a^{3\beta}a^{2\beta}a^{2\beta}a_{0\beta}a_{1\beta}a_{1\beta}


In [113]:
from itertools import combinations_with_replacement
Nocc2 = 3
Norbs2 = 6
occupiedCombinations = combinations_with_replacement(range(Nocc2), 2)
virtualCombinations = combinations_with_replacement(range(Nocc2, Norbs2), 2)
for o in occupiedCombinations:
    print(o)
for v in virtualCombinations:
    print(v)

(0, 0)
(0, 1)
(0, 2)
(1, 1)
(1, 2)
(2, 2)
(3, 3)
(3, 4)
(3, 5)
(4, 4)
(4, 5)
(5, 5)


In [114]:
print(singlesAmplitudes1)
print([o.__str__() for o in singlesExcitations1])

[[0. 0.]
 [0. 0.]]
['1.0 * a^{2\\alpha}a_{0\\alpha}\n + 1.0 * a^{2\\beta}a_{0\\beta}', '1.0 * a^{3\\alpha}a_{0\\alpha}\n + 1.0 * a^{3\\beta}a_{0\\beta}', '1.0 * a^{2\\alpha}a_{1\\alpha}\n + 1.0 * a^{2\\beta}a_{1\\beta}', '1.0 * a^{3\\alpha}a_{1\\alpha}\n + 1.0 * a^{3\\beta}a_{1\\beta}']


In [115]:
print(doublesAmplitudes1)
print([o.__str__() for o in doublesExcitations1])

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]
['1.0 * a^{2\\alpha}a^{2\\alpha}a_{0\\alpha}a_{0\\alpha}\n + 1.0 * a^{2\\alpha}a^{2\\beta}a_{0\\beta}a_{0\\alpha}\n + 1.0 * a^{2\\beta}a^{2\\alpha}a_{0\\alpha}a_{0\\beta}\n + 1.0 * a^{2\\beta}a^{2\\beta}a_{0\\beta}a_{0\\beta}', '1.0 * a^{2\\alpha}a^{3\\alpha}a_{0\\alpha}a_{0\\alpha}\n + 1.0 * a^{2\\alpha}a^{3\\beta}a_{0\\beta}a_{0\\alpha}\n + 1.0 * a^{2\\beta}a^{3\\alpha}a_{0\\alpha}a_{0\\beta}\n + 1.0 * a^{2\\beta}a^{3\\beta}a_{0\\beta}a_{0\\beta}', '1.0 * a^{3\\alpha}a^{2\\alpha}a_{0\\alpha}a_{0\\alpha}\n + 1.0 * a^{3\\alpha}a^{2\\beta}a_{0\\beta}a_{0\\alpha}\n + 1.0 * a^{3\\beta}a^{2\\alpha}a_{0\\alpha}a_{0\\beta}\n + 1.0 * a^{3\\beta}a^{2\\beta}a_{0\\beta}a_{0\\beta}', '1.0 * a^{3\\alpha}a^{3\\alpha}a_{0\\alpha}a_{0\\alpha}\n + 1.0 * a^{3\\alpha}a^{3\\beta}a_{0\\beta}a_{0\\alpha}\n + 1.0 * a^{3\\beta}a^{3\\alpha}a_{0\\alpha}a_{0\\beta}\n + 1.0 * a^{3\\beta}a^{3\\beta}a_{0\\beta}a_{0\\

In [116]:
bch = BCHSimilarityTransform(hamiltonian, getClusterDoubles(doublesAmplitudes, Nocc, Norbs), 4)
bch.checkNilpotency()
print(bch)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 

In [118]:
print(vacuumExpectationValue(bch, vacuum) + mol.energy_nuc())
print(mf.e_tot)

-0.7275948493863061
-1.1026388111917238


In [119]:
print(spinFreeDoubleExcitation(1,1,0,0).conjugate())

1.0 * a^{0\alpha}a^{0\alpha}a_{1\alpha}a_{1\alpha}
 + 1.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 1.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 1.0 * a^{0\beta}a^{0\beta}a_{1\beta}a_{1\beta}


In [120]:
projected = spinFreeDoubleExcitation(1,1,0,0).conjugate() * bch
print(projected)

0.0
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + -4.7142356915405136e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{1\beta}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{0\alpha}a^{0\beta}a_{1\b

In [122]:
vacuumExpectationValue(projected, vacuum)

4.2201869394703415

In [123]:
doublesAmplitudes[0,0,0,0] = -0.07016307

In [125]:
newbch = BCHSimilarityTransform(hamiltonian, getClusterDoubles(doublesAmplitudes, Nocc, Norbs), 4)
newbch.checkNilpotency()
excitation = spinFreeDoubleExcitation(1,1,0,0)
newprojected = (1 / sqrt(vacuumExpectationValue(excitation.conjugate() * excitation, vacuum))) * excitation.conjugate() * newbch
print(vacuumExpectationValue(newbch, vacuum))
print(vacuumExpectationValue(newprojected, vacuum, True))
#print(newprojected)


-1.7520059616031438
0.046880495225677145 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.046880495225677145 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.046880495225677145 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.046880495225677145 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.04174178612143377 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}
-0.04174178612143377 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a_{0\beta}
0.018724737791131398 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.018724737791131398 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.04174178612143377 * a^{0\al

In [126]:
print(vacuumExpectationValue(newbch, vacuum) + mol.energy_nuc())

-1.128953046936954


In [127]:
clusterDoubles = getClusterDoubles(doublesAmplitudes, Nocc, Norbs)
waveoperator = exponentialOperator(clusterDoubles, 3)
print(waveoperator)
inversewaveoperator = exponentialOperator((-1) * clusterDoubles, 3)
print(inversewaveoperator)

1.0
 + -0.07016307 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.07016307 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
1.0
 + 0.07016307 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.07016307 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


In [128]:
doubleexcitation = getClusterDoubles(np.array([[[[1.]]]]), Nocc, Norbs).summandList[1]
fcioperator = (0.990656547) + (-0.136380375) * doubleexcitation
print(fcioperator)

0.990656547
 + -0.136380375 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}


In [130]:
fcinormFactor = vacuumExpectationValue(fcioperator.conjugate() * fcioperator, vacuum, True)
print(fcinormFactor)
electronicFCIEnergy = vacuumExpectationValue(fcioperator.conjugate() * hamiltonian * fcioperator, vacuum, True) / fcinormFactor
print(electronicFCIEnergy)
print(electronicFCIEnergy + mol.energy_nuc())

0.9814003941139633
0.018599606685140626 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
1.000000000799104
-1.167719866037671 * a^{0\alpha}a_{0\alpha}
-1.167719866037671 * a^{0\beta}a_{0\beta}
0.32092259609959023 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
0.32092259609959023 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
-0.012667682818650852 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.012667682818650852 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.012667682818650852 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.012667682818650852 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.009927523359437748 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.009927523359437748 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\b

In [131]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [133]:
print(vacuumExpectationValue(inversewaveoperator * hamiltonian * waveoperator, vacuum))

-1.7520059616031438


In [134]:
normFactor = vacuumExpectationValue(waveoperator.conjugate() * waveoperator, vacuum)
electronicCCEnergy = vacuumExpectationValue(waveoperator.conjugate() * hamiltonian * waveoperator, vacuum) / normFactor
print(electronicCCEnergy)
print(electronicCCEnergy + mol.energy_nuc())

-1.7514978019778398
-1.12844488731165


In [138]:
def testDoublesAmplitudes(doublesAmplitudes_):
    newbch = BCHSimilarityTransform(hamiltonian, getClusterDoubles(doublesAmplitudes_, Nocc, Norbs), 4)
    newbch.checkNilpotency()
    excitation = spinFreeDoubleExcitation(1,1,0,0)
#    newprojected = (1 / sqrt(vacuumExpectationValue3(excitation.conjugate() * excitation, vacuum))) * excitation.conjugate() * newbch
    newprojected = excitation.conjugate() * newbch
    return vacuumExpectationValue(newprojected, vacuum)

In [139]:
def testSinglesAmplitudes(singlesAmplitudes_):
    newbch = BCHSimilarityTransform(hamiltonian, getClusterSingles(singlesAmplitudes_, Nocc, Norbs), 4)
    newbch.checkNilpotency()
    excitation = spinFreeSingleExcitation(1,0)
    newprojected = excitation.conjugate() * newbch
    return vacuumExpectationValue(newprojected, vacuum)

In [140]:
vacuumExpectationValue(hamiltonian, vacuum)

-1.7256917258579132

In [141]:
def testAmplitudes(singlesAmplitudes_, doublesAmplitudes_):
    newbch = BCHSimilarityTransform(hamiltonian, getClusterSingles(singlesAmplitudes_, Nocc, Norbs) + getClusterDoubles(doublesAmplitudes_, Nocc, Norbs), 4)
    newbch.checkNilpotency()
    excitations = [spinFreeSingleExcitation(1,0), spinFreeDoubleExcitation(1,1,0,0)]
    projectedVEVs = []
    for excitation in excitations:
        newprojected = excitation.conjugate() * newbch
        projectedVEVs.append(vacuumExpectationValue(newprojected, vacuum))
    return projectedVEVs


In [142]:
print(testDoublesAmplitudes(doublesAmplitudes))

-0.007385167335775115


In [143]:
print(singlesAmplitudes, doublesAmplitudes)
print(testAmplitudes(singlesAmplitudes, doublesAmplitudes))

[[1.]] [[[[-0.07016307]]]]
[0.9722673740422316, 0.45095247819856366]


In [144]:
from scipy.optimize import fsolve
def testFunc(x):
    doublesAmplitudes[0,0,0,0] = x
#    singlesAmplitudes[0,0] = x
    test = testDoublesAmplitudes(doublesAmplitudes)
    print(x, test)
    return test

In [145]:
amplitude = fsolve(testFunc, 0)
print(amplitude)

[0] 0.37504396180541716
[0.] 0.37504396180541716
[0.] 0.37504396180541716
[1.49011612e-08] 0.3750440414568746
[-0.07016307] -0.0073851496474540805
[-0.06880814] 0.00013986172659737056
[-0.06883332] 5.023655752880215e-08
[-0.06883333] -3.420283804442259e-13
[-0.06883333] 3.2526065174565133e-19
[-0.06883333]


In [147]:
print(amplitude[0])
doublesAmplitudes[0,0,0,0] = amplitude[0]
clusterDoubles = getClusterDoubles(doublesAmplitudes, Nocc, Norbs)
waveoperator = exponentialOperator(clusterDoubles, 3)
print(waveoperator)
inversewaveoperator = exponentialOperator((-1) * clusterDoubles, 3)
print(inversewaveoperator)
electronicCCEnergy = vacuumExpectationValue(inversewaveoperator * hamiltonian * waveoperator, vacuum)
print(electronicCCEnergy)
print(electronicCCEnergy + mol.energy_nuc())

-0.06883332874447325
1.0
 + -0.06883332874447325 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332874447325 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
1.0
 + 0.06883332874447325 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.06883332874447325 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-1.751507250174495
-1.1284543355083052


In [149]:
excitation = spinFreeDoubleExcitation(1,1,0,0)
print(excitation)
excitation.checkNilpotency()
print(excitation)
print(vacuumExpectationValue(excitation.conjugate() * excitation, vacuum))
excitedenergy = (1 / sqrt(vacuumExpectationValue(excitation.conjugate() * excitation, vacuum))) * excitation.conjugate() * hamiltonian * (1 / sqrt(vacuumExpectationValue(excitation.conjugate() * excitation, vacuum))) * excitation
print(vacuumExpectationValue(excitedenergy, vacuum))

1.0 * a^{1\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 1.0 * a^{1\beta}a^{1\beta}a_{0\beta}a_{0\beta}
0.0
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0
4.0
-0.38936201963626527


In [150]:
mf.mo_occ

NPArrayWithTag([2., 0.])

In [151]:
print(hamiltonian)

-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.09376099045135429 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 2.7755575615628914e-17 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.33906809184265285 * a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0
 + 0.3226247132942422 * a^{1\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 0.0
 + 5.204170427930421e-17 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942422 * a^{1\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 6.938893903907228e-17 * a^{1\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.093760990451354

In [152]:
eri

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [153]:
print(hamiltonian2Body)

0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3270047556780619 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
 + 5.204170427930421e-17 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{1\beta}
 + 4.163336342344337e-17 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{0\beta}
 + 0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 

In [154]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [155]:
print(hamiltonian1Body)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}


In [156]:
print(excitation)

0.0
 + 1.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 1.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [158]:
vacuumExpectationValue(0.25 * excitation.conjugate() * hamiltonian * excitation, [1,0], True)

-0.13343727541519634 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.13343727541519634 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.13343727541519634 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.13343727541519634 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.13343727541519634 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.13343727541519634 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.13343727541519634 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.13343727541519634 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{1\beta}a^{1\beta}a

-0.38936201963626527

In [160]:
vacuumExpectationValue(operatorProduct([basicOperator(1,0,0)]) * hamiltonian * operatorProduct([basicOperator(0,0,0)]), vacuum, True)

0.0 * a_{0\beta}


TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'

In [161]:
h1[1,1]

-0.5337491016607854

In [162]:
eri[2,2]

0.6781361836853057

In [163]:
2 * h1[1,1] + 1 * eri[2,2]

-0.38936201963626504

In [164]:
print(fcioperator)

0.990656547
 + -0.136380375 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}


In [165]:
a, b = 0.990656547, -0.136380375

In [166]:
h00 = 2 * h1[0,0] + 1 * eri[0,0]
h01 = 2 * h1[0,1] + 1 * eri[1,1] #Mistake is here!!!!! (eri[0,2] -> eri[1,1])
h11 = 2 * h1[1,1] + 1 * eri[2,2]

print(h00, h01, h11)

-1.7256917258579132 0.18752198090270863 -0.38936201963626504


In [167]:
fcienergy = a * a * h00 + 2 * a * b * h01 + b * b * h11

In [168]:
fcienergy + mol.energy_nuc()

-1.1284543369079416

In [170]:
vacuumExpectationValue(hamiltonian * excitation, vacuum, True)

0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


0.37504396180541716

In [171]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))